Esse código irá ler a tabela na pasta e processar para uma outra pasta. Dessa forma sempre atualizando a nova tabela apenas com itens novos.
Depois aqui mesmo, será feito uma separação da base para levar a base para o PostgreSQL.
As bases serão 5:
- dimCustomer
- dimManager
- dimProduct
- dimStore
- factOrders

# Importando as bibliotecas

In [13]:
import pandas as pd
import os
import shutil
import hashlib
import uuid
import warnings

# Criando um script para processar os dados
Esse script vai ler a tabela na fonte e salvar em uma pasta chamada **processado**, dessa forma trazendo apenas dados novos e mantendo os que já estão dentro da pasta

In [2]:
# Caminho para a pasta onde o arquivo Excel está localizado
source_file_path = 'original\\stage_MultiStore.xlsx'
# Caminho para a pasta onde o arquivo processado será movido
processed_folder = 'processado'
# Caminho para o arquivo de tabela mestre
master_file_path = 'processado\\tabela_mestre.xlsx'

# Criar a pasta processado se ela não existir
os.makedirs(processed_folder, exist_ok=True)

# Função para processar o arquivo Excel
def process_excel_file():
    # Ler o arquivo Excel original
    df_new = pd.read_excel(source_file_path)
    
    # Verificar se a tabela mestre existe
    if os.path.exists(master_file_path):
        # Ler o arquivo mestre existente
        master_df = pd.read_excel(master_file_path)
        # Identificar novas linhas baseadas na coluna 'id'
        new_rows = df_new[~df_new['Order ID'].isin(master_df['Order ID'])]
        # Adicionar as novas linhas ao arquivo mestre
        updated_master_df = pd.concat([master_df, new_rows], ignore_index=True)
    else:
        # Se não existir, o novo arquivo é o arquivo mestre
        updated_master_df = df_new
    
    # Salvar o arquivo mestre atualizado
    updated_master_df.to_excel(master_file_path, index=False)

# Executar a função
process_excel_file()


# Separando as bases e criando um arquivo csv para cada tabela nova
Essa tabela será enviada para o PostgreSQL

## Dim Customer

In [3]:
df_customer = pd.read_excel("processado/tabela_mestre.xlsx")


In [4]:
df_selected = df_customer[['Customer ID', 'Customer Name', 'Customer State']]

# Agrupar por 'Custumer ID' e manter apenas as colunas selecionadas
dimCustomer = df_selected.groupby('Customer ID').first().reset_index()

# Renomear as colunas
dimCustomer.columns = ['customerId', 'customerName', 'isActive']

dimCustomer['isActive'] = dimCustomer['isActive'].apply(lambda x: True if x == 'Active' else False)

# # Salvar o DataFrame atualizado
# df_grouped.to_excel('processado/agrupado_tabela_mestre.xlsx', index=False)

In [5]:
# salvando em formato de csv em uma pasta separada o dimCustomer 

store_split_folder = 'processado/store_split'

csv_file_path = os.path.join(store_split_folder, 'dim_customer.csv')

os.makedirs(store_split_folder, exist_ok=True)


dimCustomer.to_csv(csv_file_path, index=False)

## Dim Manager

In [6]:
df_manager = pd.read_excel("processado/tabela_mestre.xlsx")


In [7]:
df_selected = df_customer[['Regional Manager ID', 'Regional Manager']]

# Agrupar por 'Custumer ID' e manter apenas as colunas selecionadas
dimManager = df_selected.groupby('Regional Manager ID').first().reset_index()

# Renomear as colunas
dimManager.columns = ['managerId', 'managerName']

In [8]:
# salvando em formato de csv em uma pasta separada o dimManager 

store_split_folder = 'processado/store_split'

csv_file_path = os.path.join(store_split_folder, 'dim_manager.csv')

os.makedirs(store_split_folder, exist_ok=True)

dimManager.to_csv(csv_file_path, index=False)

# Dim Product
Tabela com os produtos - Não contem preço

In [9]:
df_product = pd.read_excel("processado/tabela_mestre.xlsx")

In [10]:
df_selected = df_product[['Product ID', 'Category', 'Sub-Category', 'Product Name']]

# Agrupar por 'Custumer ID' e manter apenas as colunas selecionadas
dimProduct = df_selected.groupby('Product ID').first().reset_index()

# Renomear as colunas
dimProduct.columns = ['productId', 'category', 'subCategory', 'productName']

In [11]:
# salvando em formato de csv em uma pasta separada o dimManager 

store_split_folder = 'processado/store_split'

csv_file_path = os.path.join(store_split_folder, 'dim_product.csv')

os.makedirs(store_split_folder, exist_ok=True)

dimProduct.to_csv(csv_file_path, index=False)

## Dim Store
Informações referente a localização da loja apenas

In [12]:
df_store = pd.read_excel("processado/tabela_mestre.xlsx")

In [15]:
# Selecionar as colunas desejadas
df_selected = df_store[['Country', 'State', 'City', 'Region', 'Postal Code']]


# # Remover duplicatas para garantir que cada combinação seja única
df_store = df_selected.drop_duplicates()

# Definir o prefixo e o ano atual
prefix = 'STO'

# Função para gerar um hash único baseado nos valores das colunas
def generate_unique_id(row):
    hash_input = f"{row['Country']}_{row['State']}_{row['City']}_{row['Region']}_{row['Postal Code']}"
    # Gerar um hash MD5 da string e garantir que o ID é único
    return f"{prefix}-{hashlib.md5(hash_input.encode()).hexdigest()[:6].upper()}"

# Adicionar a coluna de ID gerado com hash
df_store['storeId'] = df_store.apply(generate_unique_id, axis=1)

In [16]:
# salvando em formato de csv em uma pasta separada o dimManager 

store_split_folder = 'processado/store_split'

csv_file_path = os.path.join(store_split_folder, 'dim_store.csv')

os.makedirs(store_split_folder, exist_ok=True)

df_store.to_csv(csv_file_path, index=False)

## Fact Orders

In [22]:
df_orders = pd.read_excel("processado/tabela_mestre.xlsx")

# Join para trazer storeId
df_ordersJoin = pd.merge(df_orders, df_store, 
                     on=['Country', 'State', 'City', 'Region','Postal Code'], 
                     how='left')

In [33]:

fact_orders = df_ordersJoin[['Order ID',
                             'storeId',
                             'Order Date',
                             'Ship Date',
                             'Ship Mode',
                             'Customer ID',
                             'Segment',
                             'Regional Manager ID',
                             'Product ID',
                             'Sales',
                             'Quantity',
                             'Discount',
                             'Profit']]

# Converter números do formato serial do Excel para datetime
def excel_serial_to_datetime(serial):
    # Excel usa 1º de janeiro de 1900 como o dia 1
    return pd.Timestamp('1900-01-01') + pd.to_timedelta(serial - 2, unit='D')

fact_orders['Order Date'] = fact_orders['Order Date'].apply(excel_serial_to_datetime)
fact_orders['Ship Date'] = fact_orders['Ship Date'].apply(excel_serial_to_datetime)


C:\Users\Andre\AppData\Local\Temp\ipykernel_17692\917624760.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fact_orders['Order Date'] = fact_orders['Order Date'].apply(excel_serial_to_datetime)
C:\Users\Andre\AppData\Local\Temp\ipykernel_17692\917624760.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fact_orders['Ship Date'] = fact_orders['Ship Date'].apply(excel_serial_to_datetime)


In [34]:
renaming_dict = {
    'Order ID': 'orderId',
    'storeId': 'storeId',
    'Order Date': 'orderDate',
    'Ship Date': 'shipDate',
    'Ship Mode': 'shipMode',
    'Customer ID': 'customerId',
    'Segment': 'segment',
    'Regional Manager ID': 'managerId',
    'Product ID': 'productId',
    'Sales': 'salePrice',
    'Quantity': 'quantity',
    'Discount': 'discount',
    'Profit': 'profit'
}

# Renomear as colunas
fact_orders.rename(columns=renaming_dict, inplace=True)


C:\Users\Andre\AppData\Local\Temp\ipykernel_17692\472989207.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fact_orders.rename(columns=renaming_dict, inplace=True)


In [35]:
# salvando em formato de csv em uma pasta separada o dimManager 

store_split_folder = 'processado/store_split'

csv_file_path = os.path.join(store_split_folder, 'fact_orders.csv')

os.makedirs(store_split_folder, exist_ok=True)

fact_orders.to_csv(csv_file_path, index=False)

### Tabelas prontas. Próximo passo é mandar todas essas tabelas para o banco de dados local.

---------------------------

## Integrando o banco de dados

In [36]:
!pip install psycopg2
!pip install pandas openpyxl psycopg2-binary
!pip install --upgrade psycopg2

In [38]:
import psycopg2
from sqlalchemy import create_engine, text

In [42]:
# Definindo os parâmetros de conexão com o banco de dados
db_params = {
    'host': 'localhost',
    'database': 'postgres',  # Banco de dados padrão para criar o novo banco
    'user': 'postgres',
    'password': 'andre123'
}


In [43]:
# Create a connection to the PostgreSQL server
conn = psycopg2.connect(
    host=db_params['host'],
    database=db_params['database'],
    user=db_params['user'],
    password=db_params['password']
)

# Create a cursor object
cur = conn.cursor()

# Set automatic commit to be true, so that each action is committed without having to call conn.committ() after each command
conn.set_session(autocommit=True)

# Create the 'soccer' database
cur.execute("CREATE DATABASE stage")

# Commit the changes and close the connection to the default database
conn.commit()
cur.close()
conn.close()

In [44]:
# Connect to the 'stage' database
db_params['database'] = 'stage'
engine = create_engine(f'postgresql://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}/{db_params["database"]}')

# Define the file paths for your CSV files
csv_files = {
    'dim_customer': 'processado/store_split/dim_customer.csv',
    'dim_manager': 'processado/store_split/dim_manager.csv',
    'dim_store': 'processado/store_split/dim_store.csv',
    'dim_product': 'processado/store_split/dim_product.csv',
    'fact_orders': 'processado/store_split/fact_orders.csv'
}

# Load and display the contents of each CSV file to check
for table_name, file_path in csv_files.items():
    print(f"Contents of '{table_name}' CSV file:")
    df = pd.read_csv(file_path)
    print(df.head(2))  # Display the first few rows of the DataFrame
    print("\n")

Contents of 'dim_customer' CSV file:
  customerId  customerName  isActive
0   AA-10315    Alex Avila      True
1   AA-10375  Allen Armold      True


Contents of 'dim_manager' CSV file:
         managerId       managerName
0  EMP-2016-151022  Dianna Wasserman
1  EMP-2016-151048    Charles Kaydos


Contents of 'dim_store' CSV file:
         Country       State         City Region  Postal Code     storeId
0  United States    Kentucky    Henderson  South        42420  STO-68D97E
1  United States  California  Los Angeles   West        90036  STO-4D7F2D


Contents of 'dim_product' CSV file:
         productId   category subCategory  \
0  FUR-BO-10000112  Furniture   Bookcases   
1  FUR-BO-10000330  Furniture   Bookcases   

                                         productName  
0   Bush Birmingham Collection Bookcase, Dark Cherry  
1  Sauder Camden County Barrister Bookcase, Plank...  


Contents of 'fact_orders' CSV file:
          orderId     storeId   orderDate    shipDate      shipMode 

In [45]:
# Loop through the CSV files and import them into PostgreSQL
for table_name, file_path in csv_files.items():
    df = pd.read_csv(file_path)
    df.to_sql(table_name, engine, if_exists='replace', index=False)